In [1]:
!pip install langchain==0.1.19
!pip install langchain-openai==0.1.6
!pip install langchain-community==0.0.38

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing ins

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPEN_API_KEY')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HF_TOKEN')

In [4]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature = 0)

## Access the reviews

In [9]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [11]:
## Defining the output parser

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define the desired data structure - like a python data class
class ReviewAnalysisResponse(BaseModel):
    summary: str = Field(description="A brief summary of the customer review with maximum 3 lines")
    positives: list = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
    negatives: list = Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
    sentiment: str = Field(description="One word showing the sentiment of the review - positive, negative or neutral")
    emotions: list = Field(description="A list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="Detailed email to the customer based on the sentiment")


# Set up the parser and inject instrcution into the prompt template
parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewAnalysisResponse'>)

In [13]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Analyze the given customer review below and generate the response based on the instructions
             mentioned below in the format instructions.
             Also remember to write a detailed email response for the email field based on these conditions:
               - email should be addressed to Dear Customer and signed with Service Agent
               - thank them if the review is positive or neutral
               - apologize if the review is negative

             Format Instructions:
             {format_instructions}

             Review:
             {review}
            """


prompt = PromptTemplate(
  template = prompt_txt,
  input_variables = ["review"],
  partial_variables = {"format_instructions": parser.get_format_instructions()}
)

prompt

PromptTemplate(input_variables=['review'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"summary": {"title": "Summary", "description": "A brief summary of the customer review with maximum 3 lines", "type": "string"}, "positives": {"title": "Positives", "description": "A list showing the positives mentioned by the customer in the review if any - max 3 points", "type": "array", "items": {}}, "negatives": {"title": "Negatives", "description": "A list showing the negatives mentioned by the customer in the review if any - max 3

In [15]:
chain = (prompt
         |
         chatgpt
         |
         parser
         )

In [16]:
reviews_formatted = [{'review': review} for review in reviews]
reviews_formatted[0]

{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "}

In [17]:
responses = chain.map().invoke(reviews_formatted)

In [18]:
responses[0]

ReviewAnalysisResponse(summary='Customer is extremely satisfied with the Bluetooth speaker for beach outings', positives=['Impressively clear sound quality with just the right amount of bass', 'Waterproof feature tested true during a recent splashing incident', 'Compact size with volume that can fill the space'], negatives=[], sentiment='positive', emotions=['satisfaction', 'excitement', 'happiness'], email='Dear Customer, Thank you for your positive review! We are thrilled to hear that you are extremely satisfied with the Bluetooth speaker you ordered for beach outings. Your feedback on the sound quality, waterproof feature, compact size, and volume is greatly appreciated. We are delighted that the speaker met your expectations. If you have any further feedback or need assistance, please feel free to reach out. Thank you for choosing our product. Sincerely, Service Agent')

In [19]:
responses[0].dict()

{'summary': 'Customer is extremely satisfied with the Bluetooth speaker for beach outings',
 'positives': ['Impressively clear sound quality with just the right amount of bass',
  'Waterproof feature tested true during a recent splashing incident',
  'Compact size with volume that can fill the space'],
 'negatives': [],
 'sentiment': 'positive',
 'emotions': ['satisfaction', 'excitement', 'happiness'],
 'email': 'Dear Customer, Thank you for your positive review! We are thrilled to hear that you are extremely satisfied with the Bluetooth speaker you ordered for beach outings. Your feedback on the sound quality, waterproof feature, compact size, and volume is greatly appreciated. We are delighted that the speaker met your expectations. If you have any further feedback or need assistance, please feel free to reach out. Thank you for choosing our product. Sincerely, Service Agent'}

In [20]:
for response in responses:
  for k,v in response.dict().items():
    print(f'{k}:\n{v}')
  print('-----')
  print('\n')

summary:
Customer is extremely satisfied with the Bluetooth speaker for beach outings
positives:
['Impressively clear sound quality with just the right amount of bass', 'Waterproof feature tested true during a recent splashing incident', 'Compact size with volume that can fill the space']
negatives:
[]
sentiment:
positive
emotions:
['satisfaction', 'excitement', 'happiness']
email:
Dear Customer, Thank you for your positive review! We are thrilled to hear that you are extremely satisfied with the Bluetooth speaker you ordered for beach outings. Your feedback on the sound quality, waterproof feature, compact size, and volume is greatly appreciated. We are delighted that the speaker met your expectations. If you have any further feedback or need assistance, please feel free to reach out. Thank you for choosing our product. Sincerely, Service Agent
-----


summary:
Positive review highlighting the responsiveness, backlighting, competitive price, and swift delivery of the gaming keyboard
p

In [21]:
import pandas as pd

pd.DataFrame(response.dict() for response in responses)

,summary,positives,negatives,sentiment,emotions,email
0,Customer is extremely satisfied with the Bluet...,[Impressively clear sound quality with just th...,[],positive,"[satisfaction, excitement, happiness]","Dear Customer, Thank you for your positive rev..."
1,Positive review highlighting the responsivenes...,"[Responsiveness, Backlighting, Competitive price]",[],positive,"[Satisfaction, Excitement]","Dear Customer, Thank you for your positive rev..."
2,Customer is disappointed with the wireless ear...,[],"[Sound quality constantly cuts out, Uncomforta...",negative,"[disappointed, frustrated, dissatisfied]","Dear Customer, We are truly sorry to hear abou..."
3,Customer is disappointed with the tablet stand...,[],"[Tablet stand wobbles easily, Angles are not h...",negative,"[disappointed, frustrated, dissatisfied]","Dear Customer, We are truly sorry to hear abou..."
4,Customer is dissatisfied with the kitchen blen...,[],"[Struggles with tougher foods, Incredibly nois...",negative,"[frustration, disappointment]","Dear Customer, We are truly sorry to hear abou..."


# 2. Research Paper Analyst

In [23]:
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

In [24]:
SYS_PROMPT = """
Act as a Artificial Intelligence Expert.
Transform the input research paper abstract given below
based on the instruction input by the user.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", "{instruction}"),
    ]
)

In [25]:
chain = (prompt
            |
         chatgpt)

In [26]:
from langchain_core.messages import HumanMessage

prompt_txt = f"""
Based on the following research paper abstract,
create the summary report of maximum 10 lines
for a general audience

Abstract:
{paper_abstract}
"""
messages = [HumanMessage(content=prompt_txt)]
user_instruction = {'instruction': messages}

response = chain.invoke(user_instruction)
messages.append(response)

In [27]:
print(response.content)

Generative artificial intelligence (AI), such as ChatGPT, is raising ethical concerns, particularly in high-stakes fields like healthcare. While guidelines and regulations are being developed, gaps in ethical discussions persist. A systematic review in healthcare highlights the need for an ethics checklist to enhance transparency and trust in generative AI products. In healthcare, generative AI is used for text, image, and structured data, with a focus on resolving privacy issues and reducing bias. However, discussions on autonomy, integrity, and morality issues are lacking. The integration of ethical considerations is crucial for the responsible development and application of generative AI technologies.


In [28]:
response.content

'Generative artificial intelligence (AI), such as ChatGPT, is raising ethical concerns, particularly in high-stakes fields like healthcare. While guidelines and regulations are being developed, gaps in ethical discussions persist. A systematic review in healthcare highlights the need for an ethics checklist to enhance transparency and trust in generative AI products. In healthcare, generative AI is used for text, image, and structured data, with a focus on resolving privacy issues and reducing bias. However, discussions on autonomy, integrity, and morality issues are lacking. The integration of ethical considerations is crucial for the responsible development and application of generative AI technologies.'

In [29]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

In [31]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

In [32]:
prompt_txt = f"""
Use only the research paper abstract from earlier and create a detailed report for a healthcare company.
In the report, also include bullet points (3 max) for pros and cons of ethics in Generative AI
"""
messages.append(HumanMessage(content=prompt_txt))
user_instruction = {'instruction': messages}
response = chain.invoke(user_instruction)
messages.append(response)

In [33]:
print(response.content)

**Summary Report for Healthcare Company:**

Generative artificial intelligence (AI), exemplified by ChatGPT, is raising ethical concerns, particularly in critical fields like healthcare. While guidelines and regulations are in progress, there are existing gaps in ethical discussions. A systematic review in healthcare emphasizes the necessity of an ethics checklist to enhance transparency and trust in generative AI products. In healthcare, generative AI is utilized for text, image, and structured data, with a primary focus on addressing privacy issues and mitigating bias. However, discussions regarding autonomy, integrity, and morality issues are currently insufficient. The integration of ethical considerations is vital for the responsible development and application of generative AI technologies.

**Pros and Cons of Ethics in Generative AI:**
- **Pros:**
  - Enhances transparency and trust in AI products.
  - Helps in resolving privacy issues and reducing bias in healthcare application

In [34]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

In [35]:
prompt_txt = f"""
Use only the research paper abstract from earlier and create a detailed report for a generative AI company solving healthcare problems.
In the report also include sections for key points mentioned around Generative AI for text, images and structured data based healthcare
"""
messages.append(HumanMessage(content=prompt_txt))
user_instruction = {'instruction': messages}
response = chain.invoke(user_instruction)

In [36]:
print(response.content)

**Summary Report for Generative AI Company in Healthcare Solutions:**

Generative artificial intelligence (AI), exemplified by ChatGPT, is raising ethical concerns, particularly in critical fields like healthcare. While guidelines and regulations are in progress, there are existing gaps in ethical discussions. A systematic review in healthcare emphasizes the necessity of an ethics checklist to enhance transparency and trust in generative AI products. In healthcare, generative AI is utilized for text, image, and structured data, with a primary focus on addressing privacy issues and mitigating bias. However, discussions regarding autonomy, integrity, and morality issues are currently insufficient. The integration of ethical considerations is vital for the responsible development and application of generative AI technologies.

**Key Points on Generative AI in Healthcare:**
- **Text Data:** Ethical considerations are discussed extensively, particularly focusing on issues caused by large la

# 3. Social Media Marketing Analyst

In [37]:
fact_sheet_mobile = """
PRODUCT NAME
Samsung Galaxy Z Fold4 5G Black
​
PRODUCT OVERVIEW
Stands out. Stands up. Unfolds.
The Galaxy Z Fold4 does a lot in one hand with its 15.73 cm(6.2-inch) Cover Screen.
Unfolded, the 19.21 cm(7.6-inch) Main Screen lets you really get into the zone.
Pushed-back bezels and the Under Display Camera means there's more screen
and no black dot getting between you and the breathtaking Infinity Flex Display.
Do more than more with Multi View. Whether toggling between texts or catching up
on emails, take full advantage of the expansive Main Screen with Multi View.
PC-like power thanks to Qualcomm Snapdragon 8+ Gen 1 processor in your pocket,
transforms apps optimized with One UI to give you menus and more in a glance
New Taskbar for PC-like multitasking. Wipe out tasks in fewer taps. Add
apps to the Taskbar for quick navigation and bouncing between windows when
you're in the groove.4 And with App Pair, one tap launches up to three apps,
all sharing one super-productive screen
Our toughest Samsung Galaxy foldables ever. From the inside out,
Galaxy Z Fold4 is made with materials that are not only stunning,
but stand up to life's bumps and fumbles. The front and rear panels,
made with exclusive Corning Gorilla Glass Victus+, are ready to resist
sneaky scrapes and scratches. With our toughest aluminum frame made with
Armor Aluminum, this is one durable smartphone.
World’s first water resistant foldable smartphones. Be adventurous, rain
or shine. You don't have to sweat the forecast when you've got one of the
world's first water-resistant foldable smartphones.
​
PRODUCT SPECS
OS - Android 12.0
RAM - 12 GB
Product Dimensions - 15.5 x 13 x 0.6 cm; 263 Grams
Batteries - 2 Lithium Ion batteries required. (included)
Item model number - SM-F936BZKDINU_5
Wireless communication technologies - Cellular
Connectivity technologies - Bluetooth, Wi-Fi, USB, NFC
GPS - True
Special features - Fast Charging Support, Dual SIM, Wireless Charging, Built-In GPS, Water Resistant
Other display features - Wireless
Device interface - primary - Touchscreen
Resolution - 2176x1812
Other camera features - Rear, Front
Form factor - Foldable Screen
Colour - Phantom Black
Battery Power Rating - 4400
Whats in the box - SIM Tray Ejector, USB Cable
Manufacturer - Samsung India pvt Ltd
Country of Origin - China
Item Weight - 263 g
"""

In [39]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
​
Write a brief product description

Technical specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)

In [40]:
chain = (chat_template
            |
         chatgpt)
response = chain.invoke({"fact_sheet_mobile": fact_sheet_mobile})

In [41]:
print(response.content)

Introducing the Samsung Galaxy Z Fold4 5G Black - the ultimate smartphone that stands out, stands up, and unfolds to reveal a world of possibilities. With a stunning 6.2-inch Cover Screen and a massive 7.6-inch Main Screen, you can truly immerse yourself in your favorite content like never before.

Powered by the Qualcomm Snapdragon 8+ Gen 1 processor and 12GB of RAM, the Galaxy Z Fold4 offers PC-like power in the palm of your hand. The new Taskbar feature allows for seamless multitasking, while the App Pair function lets you launch up to three apps simultaneously on one screen.

But that's not all - the Galaxy Z Fold4 is our toughest foldable smartphone yet, featuring Corning Gorilla Glass Victus+ panels and an Armor Aluminum frame for added durability. Plus, it's the world's first water-resistant foldable smartphone, so you can take it with you wherever you go, rain or shine.

Experience the future of smartphones with the Samsung Galaxy Z Fold4 5G Black. Order yours today and elevate

In [42]:
from IPython.display import display, Markdown
display(Markdown(response.content))

Introducing the Samsung Galaxy Z Fold4 5G Black - the ultimate smartphone that stands out, stands up, and unfolds to reveal a world of possibilities. With a stunning 6.2-inch Cover Screen and a massive 7.6-inch Main Screen, you can truly immerse yourself in your favorite content like never before.

Powered by the Qualcomm Snapdragon 8+ Gen 1 processor and 12GB of RAM, the Galaxy Z Fold4 offers PC-like power in the palm of your hand. The new Taskbar feature allows for seamless multitasking, while the App Pair function lets you launch up to three apps simultaneously on one screen.

But that's not all - the Galaxy Z Fold4 is our toughest foldable smartphone yet, featuring Corning Gorilla Glass Victus+ panels and an Armor Aluminum frame for added durability. Plus, it's the world's first water-resistant foldable smartphone, so you can take it with you wherever you go, rain or shine.

Experience the future of smartphones with the Samsung Galaxy Z Fold4 5G Black. Order yours today and elevate your mobile experience to new heights.

In [43]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
​
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen, processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put these features in the table - foldable display screen, processing power, RAM, camera and battery life

Technical specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)

In [44]:
chain = (chat_template
            |
         chatgpt)
response = chain.invoke({"fact_sheet_mobile": fact_sheet_mobile})

In [45]:
print(response.content)

Product Name: Samsung Galaxy Z Fold4 5G Black

Description: The Samsung Galaxy Z Fold4 5G Black is a revolutionary foldable smartphone that combines cutting-edge technology with a sleek design. With a 6.2-inch Cover Screen and a 7.6-inch Main Screen, you can easily switch between tasks and enjoy a truly immersive experience. Powered by a Qualcomm Snapdragon 8+ Gen 1 processor and 12GB of RAM, this phone offers PC-like power in the palm of your hand. The camera features on this device are top-notch, allowing you to capture stunning photos and videos. Plus, with a 4400mAh battery, you can stay connected all day long.

Product Specifications:

| Feature                  | Specification          |
|--------------------------|------------------------|
| Foldable Display Screen  | 6.2-inch Cover Screen, 7.6-inch Main Screen |
| Processing Power         | Qualcomm Snapdragon 8+ Gen 1 processor |
| RAM                      | 12 GB                  |
| Camera                   | Rear and Front 

In [46]:
from IPython.display import display, Markdown
display(Markdown(response.content))

Product Name: Samsung Galaxy Z Fold4 5G Black

Description: The Samsung Galaxy Z Fold4 5G Black is a revolutionary foldable smartphone that combines cutting-edge technology with a sleek design. With a 6.2-inch Cover Screen and a 7.6-inch Main Screen, you can easily switch between tasks and enjoy a truly immersive experience. Powered by a Qualcomm Snapdragon 8+ Gen 1 processor and 12GB of RAM, this phone offers PC-like power in the palm of your hand. The camera features on this device are top-notch, allowing you to capture stunning photos and videos. Plus, with a 4400mAh battery, you can stay connected all day long.

Product Specifications:

| Feature                  | Specification          |
|--------------------------|------------------------|
| Foldable Display Screen  | 6.2-inch Cover Screen, 7.6-inch Main Screen |
| Processing Power         | Qualcomm Snapdragon 8+ Gen 1 processor |
| RAM                      | 12 GB                  |
| Camera                   | Rear and Front         |
| Battery Life             | 4400mAh                |

In [47]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a
description for a retail website advert of a product based
on a technical fact sheet specifications for a mobile smartphone
​
Write a catchy product description with some emojis,
which uses at most 60 words
and focuses on the most important things about the smartphone
which might matter to users like display and camera

Technical specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)

In [48]:
chain = (chat_template
            |
         chatgpt)
response = chain.invoke({"fact_sheet_mobile": fact_sheet_mobile})

In [49]:
print(response.content)

🌟 Introducing the Samsung Galaxy Z Fold4 5G Black 🌟 Unfold endless possibilities with a 7.6-inch Main Screen and Under Display Camera for a seamless viewing experience. Powered by Qualcomm Snapdragon 8+ Gen 1 processor, multitask like a pro with Multi View and Taskbar. With water-resistant and durable design, this foldable smartphone is ready for any adventure. Get yours now and elevate your mobile experience! 📱🌈 #SamsungGalaxyZFold4 #InnovationUnfolded


In [50]:
from IPython.display import display, Markdown
display(Markdown(response.content))

🌟 Introducing the Samsung Galaxy Z Fold4 5G Black 🌟 Unfold endless possibilities with a 7.6-inch Main Screen and Under Display Camera for a seamless viewing experience. Powered by Qualcomm Snapdragon 8+ Gen 1 processor, multitask like a pro with Multi View and Taskbar. With water-resistant and durable design, this foldable smartphone is ready for any adventure. Get yours now and elevate your mobile experience! 📱🌈 #SamsungGalaxyZFold4 #InnovationUnfolded

# 4. IT Support Analysts

In [51]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define your desired data structure - like a python data class.
class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description="The original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description="Response to the customer in their original language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")


parser = JsonOutputParser(pydantic_object=ITSupportResponse)

In [52]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Act as an Information Technology (IT) customer support agent.
             For the IT support message mentioned below
             use the following output format when generating the output response

             Output format instructions:
             {format_instructions}

             Customer IT support message:
             {it_support_msg}
             """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["it_support_msg"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [53]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
llm_chain = (prompt
              |
            chatgpt
              |
            parser)

In [54]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

formatted_msgs = [{"it_support_msg": msg}
                    for msg in it_support_queue]
formatted_msgs[0]

{'it_support_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.'}

In [55]:
responses = llm_chain.map().invoke(formatted_msgs)

In [56]:
responses[0]

{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.',
 'orig_lang': 'Portuguese',
 'category': 'Sync Issue',
 'trans_msg': "I can't sync my contacts with the phone. I always get a failure message.",
 'response': 'Por favor, verifique se a conexão Bluetooth está ativada no telefone e se os contatos estão configurados para sincronização. Se o problema persistir, tente reiniciar o telefone.',
 'trans_response': 'Please check if Bluetooth connection is enabled on the phone and if contacts are set up for synchronization. If the issue persists, try restarting the phone.'}

In [57]:
type(responses[0])

dict

In [58]:
import pandas as pd

df = pd.DataFrame(responses)
df

,orig_msg,orig_lang,category,trans_msg,response,trans_response
0,Não consigo sincronizar meus contatos com o te...,Portuguese,Sync Issue,I can't sync my contacts with the phone. I alw...,"Por favor, verifique se a conexão Bluetooth es...",Please check if Bluetooth connection is enable...
1,Ho problemi a stampare i documenti da remoto. ...,Italian,Printing,I have problems printing documents remotely. T...,Grazie per averci contattato. Per risolvere il...,Thank you for contacting us. To resolve the is...
2,プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。,Japanese,Printer,"I replaced the toner of the printer, but the p...",トナーを正しく交換したことを確認してください。プリンターのメンテナンスを行うことで印刷品質が...,Please make sure you replaced the toner correc...
3,"Я не могу войти в систему учета времени, появл...",Russian,Login Issue,"I can't log into the time tracking system, an ...","Пожалуйста, попробуйте сбросить пароль и попро...",Please try resetting your password and attempt...
4,Internet bağlantım çok yavaş ve bazen tamamen ...,Turkish,Internet Connectivity,My internet connection is very slow and someti...,"Evet, tabii ki yardımcı olabiliriz. Lütfen mod...","Yes, of course we can help. Please check your ..."
5,Не могу установить обновление безопасности. По...,Russian,Installation,I can't install the security update. An error ...,"Извините за возникшие проблемы. Пожалуйста, по...",Apologies for the inconvenience. Please try th...
